In [2]:
def generate_moose_d_function(a, b, r1, r2, delta, angle_ranges_deg):
    import math

    # 基礎 shape：限制在 radial range 內的區域
    sector_expr = f"(tanh((sqrt(x^2/{a}^2 + y^2/{b}^2) - {r1})/{delta}) - tanh((sqrt(x^2/{a}^2 + y^2/{b}^2) - {r2})/{delta})) / 2"

    def nested_if(depth):
        if depth >= len(angle_ranges_deg):
            return "0"
        theta_min = math.radians(angle_ranges_deg[depth][0])
        theta_max = math.radians(angle_ranges_deg[depth][1])
        return (
            f"if(atan2(y,x) >= {theta_min},\n"
            f"  if(atan2(y,x) <= {theta_max},\n"
            f"    {sector_expr},\n"
            f"    {nested_if(depth + 1)}),\n"
            f"  {nested_if(depth + 1)})"
        )

    full_expr = nested_if(0)

    block = f"""[./d_multi_sector]
  type = ParsedFunction
  expression = '{full_expr}'
[../]"""

    print(block)


In [4]:
generate_moose_d_function(
    a=4.0,
    b=1.0,
    r1=0.97,
    r2=0.975,
    delta=0.002,
    angle_ranges_deg=[(0,90)]
)


[./d_multi_sector]
  type = ParsedFunction
  expression = 'if(atan2(y,x) >= 0.0,
  if(atan2(y,x) <= 1.5707963267948966,
    (tanh((sqrt(x^2/4.0^2 + y^2/1.0^2) - 0.97)/0.002) - tanh((sqrt(x^2/4.0^2 + y^2/1.0^2) - 0.975)/0.002)) / 2,
    0),
  0)'
[../]


In [7]:
import numpy as np

def generate_rotated_cracks_on_quarter_ellipse(a, b, num_cracks, theta_min_deg=15, theta_max_deg=85):
    thetas = np.linspace(np.radians(theta_min_deg), np.radians(theta_max_deg), num_cracks)
    cx_list = []
    cy_list = []
    angle_z_list = []

    for theta in thetas:
        x = a * np.cos(theta)
        y = b * np.sin(theta)

        # 法線方向：垂直於切線方向
        dx_dtheta = -a * np.sin(theta)
        dy_dtheta =  b * np.cos(theta)
        angle_rad = np.arctan2(dy_dtheta, dx_dtheta) + np.pi/2  # 正常方向（垂直切線）

        cx_list.append(f"{x:.3f}")
        cy_list.append(f"{y:.3f}")
        angle_z_list.append(f"{np.degrees(angle_rad):.1f}")

    print("[ICs]\n  [init_d_box]")
    print("    type = MultiRotBoundingBoxIC")
    print("    variable = d")
    print(f"    cx = '{' '.join(cx_list)}'")
    print(f"    cy = '{' '.join(cy_list)}'")
    print("    lx = '" + " ".join(["0.15"] * num_cracks) + "'")
    print("    ly = '" + " ".join(["0.01"] * num_cracks) + "'")
    print(f"    angle_z = '{' '.join(angle_z_list)}'")
    print("    inside = '" + " ".join(["1"] * num_cracks) + "'")
    print("    outside = '" + " ".join(["0"] * num_cracks) + "'")
    print("    int_width = '" + " ".join(["0.001"] * num_cracks) + "'")
    print("  [../]\n[]")

# 使用範例
generate_rotated_cracks_on_quarter_ellipse(a=4.0, b=1.0, num_cracks=12, theta_min_deg=5, theta_max_deg=85)


[ICs]
  [init_d_box]
    type = MultiRotBoundingBoxIC
    variable = d
    cx = '3.985 3.909 3.770 3.570 3.313 3.002 2.643 2.242 1.805 1.338 0.850 0.349'
    cy = '0.087 0.213 0.335 0.451 0.561 0.661 0.751 0.828 0.892 0.942 0.977 0.996'
    lx = '0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15 0.15'
    ly = '0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01 0.01'
    angle_z = '199.3 221.0 234.8 243.7 249.7 254.1 257.6 260.4 262.8 264.9 266.9 268.7'
    inside = '1 1 1 1 1 1 1 1 1 1 1 1'
    outside = '0 0 0 0 0 0 0 0 0 0 0 0'
    int_width = '0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001 0.001'
  [../]
[]
